# Spatial Joins Exercises

Here\'s a reminder of some of the functions we have seen. Hint: they
should be useful for the exercises!

-   `sum(expression)`: aggregate to
    return a sum for a set of records
-   `count(expression)`: aggregate to
    return the size of a set of records
-   `ST_Area(geometry)` returns the
    area of the polygons
-   `ST_AsText(geometry)` returns WKT `text`
-   `ST_Contains(geometry A, geometry B)` returns the true if geometry A contains geometry B
-   `ST_Distance(geometry A, geometry B)` returns the minimum distance between geometry A and
    geometry B
-   `ST_DWithin(geometry A, geometry B, radius)` returns the true if geometry A is radius distance or less from geometry B
-   `ST_GeomFromText(text)` returns `geometry`
-   `ST_Intersects(geometry A, geometry B)` returns the true if geometry A intersects geometry B
-   `ST_Length(linestring)` returns the length of the linestring
-   `ST_Touches(geometry A, geometry B)` returns the true if the boundary of geometry A touches geometry B
-   `ST_Within(geometry A, geometry B)` returns the true if geometry A is within geometry B


Uncomment and run the following cell to install the required packages.


In [18]:
%pip install duckdb leafmap
%pip install folium matplotlib mapclassify

Download the [nyc_data.zip](https://github.com/opengeos/data/raw/main/duckdb/nyc_data.zip) dataset using leafmap. The zip file contains the following datasets. Create a new DuckDB database and import the datasets into the database. Each dataset should be imported into a separate table.

- nyc_census_blocks
- nyc_homicides
- nyc_neighborhoods
- nyc_streets
- nyc_subway_stations

In [19]:
import leafmap
import duckdb

# 下载并解压 ZIP 文件
url = "https://open.gishub.org/data/duckdb/nyc_data.zip"
leafmap.download_file(url, unzip=True, overwrite=True)

# 连接 DuckDB 并启用 httpfs
con = duckdb.connect("nyc_data.db")
con.execute("INSTALL httpfs;")
con.execute("LOAD httpfs;")
con.install_extension("spatial")
con.load_extension("spatial")

con.sql("SHOW TABLES;")


Downloading...
From: https://open.gishub.org/data/duckdb/nyc_data.zip
To: /content/nyc_data.zip
100%|██████████| 8.73M/8.73M [00:00<00:00, 78.4MB/s]


Extracting files...


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ nyc_neighborhoods   │
│ nyc_streets         │
│ nyc_subway_stations │
└─────────────────────┘

In [20]:
import os

extract_path = "/content/nyc_data"

# 检查路径是否存在
if os.path.exists(extract_path):
    if os.path.isfile(extract_path):  # 该路径是一个文件
        print("⚠️ `/content/nyc_data` 已经是一个文件，先删除")
        os.remove(extract_path)  # 删除该文件
    elif os.path.isdir(extract_path):  # 该路径是一个目录
        print("✅ `/content/nyc_data` 已经是一个文件夹")
else:
    print("🔍 `/content/nyc_data` 目录不存在，可以安全解压")

✅ `/content/nyc_data` 已经是一个文件夹


In [21]:
import zipfile

zip_path = "/content/nyc_data.zip"
extract_path = "/content/nyc_data"

# 重新解压 ZIP 文件
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# 查看解压后的内容
print("📂 解压后文件列表:", os.listdir(extract_path))

📂 解压后文件列表: ['nyc_census_blocks.dbf', 'nyc_homicides.dbf', 'nyc_neighborhoods.prj', 'nyc_neighborhoods.shp', 'nyc_streets.dbf', 'nyc_homicides.shx', 'nyc_streets.shx', 'nyc_subway_stations.prj', 'nyc_census_sociodata.sql', 'nyc_homicides.shp', 'nyc_subway_stations.dbf', 'nyc_streets.shp', 'nyc_census_blocks.shp', 'nyc_census_blocks.shx', 'README.txt', 'nyc_streets.prj', 'nyc_neighborhoods.dbf', 'nyc_subway_stations.shx', 'nyc_neighborhoods.shx', 'nyc_census_blocks.prj', 'nyc_homicides.prj', 'nyc_subway_stations.shp']


**Create tables which could be used afterwards.**

In [24]:
shp_file = "/content/nyc_data/nyc_neighborhoods.shp"
shp_file1 = "/content/nyc_data/nyc_subway_stations.shp"
shp_file2 = "/content/nyc_data/nyc_streets.shp"
shp_file3 = "/content/nyc_data/nyc_census_block.shp"

# con.execute(f"""
#     CREATE TABLE nyc_streets AS
#     SELECT * FROM ST_Read('{shp_file}');
# """)

print(con.execute("SHOW TABLES;").fetchdf())
print(con.execute("SELECT * FROM nyc_subway_stations LIMIT 5").fetchdf())

                  name
0    nyc_neighborhoods
1          nyc_streets
2  nyc_subway_stations
   OBJECTID     ID          NAME         ALT_NAME         CROSS_ST  \
0       1.0  376.0  Cortlandt St             None        Church St   
1       2.0    2.0     Rector St             None             None   
2       3.0    1.0   South Ferry             None             None   
3       4.0  125.0      138th St  Grand Concourse  Grand Concourse   
4       5.0  126.0      149th St  Grand Concourse  Grand Concourse   

                                LONG_NAME                             LABEL  \
0            Cortlandt St (R,W) Manhattan                Cortlandt St (R,W)   
1                 Rector St (1) Manhattan                     Rector St (1)   
2               South Ferry (1) Manhattan                   South Ferry (1)   
3  138th St / Grand Concourse (4,5) Bronx  138th St / Grand Concourse (4,5)   
4    149th St / Grand Concourse (4) Bronx    149th St / Grand Concourse (4)   

     BOROUGH

1. **What subway station is in \'Little Italy\'? What subway route is it on?**

In [25]:
con.sql("SELECT s.name AS subway_station FROM nyc_neighborhoods AS n JOIN nyc_subway_stations AS s ON ST_Contains(n.geom, s.geom) WHERE n.name = 'Little Italy';")

┌────────────────┐
│ subway_station │
│    varchar     │
├────────────────┤
│ Spring St      │
└────────────────┘

In [27]:
italy_station_df = con.sql("SELECT s.name AS subway_station, s.routes as subway_route, st_astext(s.geom) as geometry FROM nyc_neighborhoods AS n JOIN nyc_subway_stations AS s ON ST_Contains(n.geom, s.geom) WHERE n.name = 'Little Italy';").df()
italy_station_df.head()

,subway_station,subway_route,geometry
0,Spring St,6,POINT (584696.3146033001 4508413.075791954)


In [28]:
italy_station_df = leafmap.df_to_gdf(italy_station_df, src_crs="EPSG:26918", dst_crs="EPSG:4326")
italy_station_df.head()

,subway_station,subway_route,geometry
0,Spring St,6,POINT (-73.99713 40.72229)


In [29]:
italy_station_df.explore()

Spring St is in " Little Italy" and subway route 6 is in Little Italy.

2. **What are all the neighborhoods served by the 6-train?** (Hint: The `routes` column in the `nyc_subway_stations` table has values like \'B,D,6,V\' and \'C,6\')


In [31]:
italy_station_df = con.sql("SELECT distinct on (n.name) n.name AS neighborhoods, s.routes as subway_route, st_astext(s.geom) as geometry FROM nyc_neighborhoods AS n JOIN nyc_subway_stations AS s ON ST_Contains(n.geom, s.geom) WHERE s.routes = '6';").df()
italy_station_df.head()

,neighborhoods,subway_route,geometry
0,Chinatown,6,POINT (584414.3752830802 4507981.764333538)
1,Greenwich Village,6,POINT (584901.3978172147 4508818.983088133)
2,Midtown,6,POINT (586780.4926467581 4512304.301414792)
3,Upper East Side,6,POINT (587430.060968678 4513505.55859977)
4,Yorkville,6,POINT (588783.3683566097 4516014.70932135)


The neighborhoods served by the 6-train are Chinatown, Greenwich Village, Midtown and Upper East Side as well as Yorkville.

3. **After 9/11, the \'Battery Park\' neighborhood was off limits for several days. How many people had to be evacuated?**

4. **What neighborhood has the highest population density (persons/km2)?**


When you're finished, you can check your answers [here](https://postgis.net/workshops/postgis-intro/joins_exercises.html).

# Ship-to-Ship Transfer Detection

Now for a less structured exercise. We're going to look at ship-to-ship transfers. The idea is that two ships meet up in the middle of the ocean, and one ship transfers cargo to the other. This is a common way to avoid sanctions, and is often used to transfer oil from sanctioned countries to other countries. We're going to look at a few different ways to detect these transfers using AIS data.

In [ ]:
%pip install duckdb duckdb-engine jupysql

In [ ]:
import duckdb
import pandas as pd

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%sql duckdb:///:memory:

In [ ]:
%%sql
INSTALL httpfs;
LOAD httpfs;
INSTALL spatial;
LOAD spatial;

## Step 1

Create a spatial database using the following AIS data:

https://storage.googleapis.com/qm2/casa0025_ships.csv

Each row in this dataset is an AIS 'ping' indicating the position of a ship at a particular date/time, alongside vessel-level characteristics.

It contains the following columns:
* `vesselid`: A unique numerical identifier for each ship, like a license plate
* `vessel_name`: The ship's name
* `vsl_descr`: The ship's type
* `dwt`: The ship's Deadweight Tonnage (how many tons it can carry)
* `v_length`: The ship's length in meters
* `draught`: How many meters deep the ship is draughting (how low it sits in the water). Effectively indicates how much cargo the ship is carrying
* `sog`: Speed over Ground (in knots)
* `date`: A timestamp for the AIS signal
* `lat`: The latitude of the AIS signal (EPSG:4326)
* `lon`: The longitude of the AIS signal (EPSG:4326)

Create a table called 'ais' where each row is a different AIS ping, with no superfluous information. Construct a geometry column.

Create a second table called 'vinfo' which contains vessel-level information with no superfluous information.

## Step 2

Use a spatial join to identify ship-to-ship transfers in this dataset.
Two ships are considered to be conducting a ship to ship transfer IF:

* They are within 500 meters of each other
* For more than two hours
* And their speed is lower than 1 knot

Some things to consider: make sure you're not joining ships with themselves. Try working with subsets of the data first while you try different things out.